In [6]:
import polars as pl
import numpy as np
from datetime import datetime,timedelta
import random

# Pytorch for ML

import torch
import torch.nn as nn
import torch.optim as optim
import research

#visualization
import altair as alt

#data
import binance

import importlib
importlib.reload(research)

<module 'research' from '/Users/ujjawalpratapsingh/Documents/quant-strategy/research.py'>

In [3]:
research.set_seed(42)

In [5]:
pl.Config.set_tbl_width_chars(200)
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_cols(-1)

polars.config.Config

In [7]:
sym = 'BTCUSDT'

hist_data_window = 7 * 4 * 6

time_interval = '1h'

max_lags = 4

forecast_horizon = 1

annualized_rate = research.sharpe_annualization_factor(time_interval, 365,24)